In [1]:
import os
import ctypes
from gtimer import GlobalTimer
from numpy.ctypeslib import ndpointer
import numpy as np

PLAN_LIB_PATH = "../cmake-build-debug/libmoveit_plan_compact.so"

from enum import Enum

class ObjectType(Enum):
    BOX = 1
    SPHERE = 2
    CYLINDER = 3

class ObjectAction(Enum):
    ADD = 0
    REMOVE = 1
    APPEND = 2
    MOVE = 3

clib = ctypes.cdll.LoadLibrary(PLAN_LIB_PATH)
MAX_STR_LEN = clib.get_max_str_len()
MAX_NAME_LEN = clib.get_max_name_len()
MAX_JOINT_NUM = clib.get_max_joint_num()
MAX_TRAJ_LEN = clib.get_max_traj_len()

class c_string(ctypes.Structure):
    _fields_ = [("buffer", ctypes.c_char*MAX_STR_LEN),
                ("len", ctypes.c_int)
                ]

class c_trajectory(ctypes.Structure):
    _fields_ = [("joints", ctypes.c_double*(MAX_JOINT_NUM*MAX_TRAJ_LEN)),
                ("name_len", ctypes.c_int),
                ("traj_len", ctypes.c_int),
                ("success", ctypes.c_bool)
                ]

class c_plan_request(ctypes.Structure):
    _fields_ = [("group_name", ctypes.c_char*MAX_NAME_LEN),
                ("tool_link", ctypes.c_char*MAX_NAME_LEN),
                ("goal_link", ctypes.c_char*MAX_NAME_LEN),
                ("init_state", ctypes.c_double*MAX_JOINT_NUM),
                ("goal_pose", ctypes.c_double*7),
                ("timeout", ctypes.c_double)
                ]

class c_object_msg(ctypes.Structure):
    _fields_ = [("name", ctypes.c_char*MAX_NAME_LEN),
                ("link_name", ctypes.c_char*MAX_NAME_LEN),
                ("pose", ctypes.c_double*7),
                ("dims", ctypes.c_double*3),
                ("type", ctypes.c_int),
                ("action", ctypes.c_int)
                ]

clib.hello_cstr.restype = c_string
clib.hello_char.restype = ndpointer(dtype=ctypes.c_char, shape=(MAX_STR_LEN,))
clib.init_planner.restype = c_string
clib.plan_compact.restype = c_trajectory
clib.plan_compact.argtypes = [c_plan_request]
clib.process_object.argtypes = [c_object_msg]

def convert_trajectory(traj, joint_num):
    joints = []
    for i_traj in range(traj.traj_len):
        joints.append(traj.joints[i_traj*MAX_JOINT_NUM:i_traj*MAX_JOINT_NUM+joint_num])
    return np.array(joints), traj.success


In [2]:
gtimer = GlobalTimer.instance()

In [3]:
urdf_path = "../test_assets/custom_robots.urdf"
srdf_path = "../test_assets/custom_robots.srdf"

urdf_str = c_string()
srdf_str = c_string()
with open(urdf_path, 'r') as f:
    while True:
        line = f.readline()
        urdf_str.buffer += line
        if not line: break
            
urdf_str.buffer
with open(srdf_path, 'r') as f:
    while True:
        line = f.readline()
        srdf_str.buffer += line
        if not line: break

In [4]:
gtimer.reset(1e3,'ms')
gtimer.tic("init_ros")
c_joint_names = clib.init_planner(urdf_str, srdf_str)
joint_names = c_joint_names.buffer.split()
joint_num = c_joint_names.len
gtimer.toc("init_ros")

88.40298652648926

In [5]:
gtimer.tic("add_object")
omsg = c_object_msg()
omsg.name = "box"
omsg.link_name = "base_link"
goal_pose = [-0.3,-0.2,0.0,0,0,0,1]
for igp in range(7): omsg.pose[igp] = goal_pose[igp]
dims = [0.1,0.1,0.1]
for igp in range(3): omsg.dims[igp] = dims[igp]
omsg.action = ObjectAction.ADD.value
omsg.type = ObjectType.BOX.value
clib.process_object(omsg)
gtimer.toc("add_object")

0.26488304138183594

In [6]:
GROUP_NAME = 'indy0'
TOOL_LINK = "indy0_tcp"
GOAL_LINK = "base_link"
JOINT_HOME = [0, 0, -np.pi / 2, 0, -np.pi / 2, 0, 0, -np.pi / 8, 0, -np.pi / 2, 0, np.pi / 2, np.pi / 2]

In [7]:
for _ in range(10):
    gtimer.tic("plan_compact")
    goal = c_plan_request()
    goal.group_name = GROUP_NAME
    goal.tool_link = TOOL_LINK
    goal_pose = [-0.3, -0.2, 0.4, 0, 0, 0, 1]
    for igp in range(7): goal.goal_pose[igp] = goal_pose[igp]
    for iis in range(joint_num): goal.init_state[iis] = JOINT_HOME[iis]
    goal.goal_link = GOAL_LINK
    goal.timeout = 0.1
    c_traj = clib.plan_compact(goal)
    gtimer.toc("plan_compact")
    gtimer.tic("convert_trajectory")
    traj, succ = convert_trajectory(c_traj, joint_num)
    gtimer.toc("convert_trajectory")
print(gtimer)
print(succ)

init_ros: 	88.0 ms/1 = 88.403 ms (88.403/88.403)
add_object: 	0.0 ms/1 = 0.265 ms (0.265/0.265)
plan_compact: 	462.0 ms/10 = 46.174 ms (33.499/72.59)
convert_trajectory: 	0.0 ms/10 = 0.042 ms (0.033/0.062)

True


In [10]:
gtimer.tic("clear_objects")
clib.clear_all_objects()
gtimer.toc("clear_objects")

0.08487701416015625

In [13]:
for _ in range(10):
    gtimer.tic("plan_compact")
    goal = c_plan_request()
    goal.group_name = GROUP_NAME
    goal.tool_link = TOOL_LINK
    goal_pose = [-0.3, -0.2, 0.4, 0, 0, 0, 1]
    for igp in range(7): goal.goal_pose[igp] = goal_pose[igp]
    for iis in range(joint_num): goal.init_state[iis] = JOINT_HOME[iis]
    goal.goal_link = GOAL_LINK
    goal.timeout = 0.1
    c_traj = clib.plan_compact(goal)
    gtimer.toc("plan_compact")
    gtimer.tic("convert_trajectory")
    traj, succ = convert_trajectory(c_traj, joint_num)
    gtimer.toc("convert_trajectory")
print(gtimer)
print(succ)

init_ros: 	88.0 ms/1 = 88.403 ms (88.403/88.403)
add_object: 	0.0 ms/1 = 0.265 ms (0.265/0.265)
plan_compact: 	1146.0 ms/22 = 52.086 ms (27.923/139.322)
convert_trajectory: 	1.0 ms/20 = 0.044 ms (0.033/0.062)
clear_objects: 	0.0 ms/1 = 0.085 ms (0.085/0.085)

True


In [14]:
gtimer.tic("add_object2")
omsg = c_object_msg()
omsg.name = "box"
omsg.link_name = "base_link"
goal_pose = [-0.3,-0.2,0.4,0,0,0,1]
for igp in range(7): omsg.pose[igp] = goal_pose[igp]
dims = [0.1,0.1,0.1]
for igp in range(3): omsg.dims[igp] = dims[igp]
omsg.action = ObjectAction.ADD.value
omsg.type = ObjectType.BOX.value
clib.process_object(omsg)
gtimer.toc("add_object2")

0.3020763397216797

In [15]:
for _ in range(10):
    gtimer.tic("plan_compact")
    goal = c_plan_request()
    goal.group_name = GROUP_NAME
    goal.tool_link = TOOL_LINK
    goal_pose = [-0.3, -0.2, 0.4, 0, 0, 0, 1]
    for igp in range(7): goal.goal_pose[igp] = goal_pose[igp]
    for iis in range(joint_num): goal.init_state[iis] = JOINT_HOME[iis]
    goal.goal_link = GOAL_LINK
    goal.timeout = 0.1
    c_traj = clib.plan_compact(goal)
    gtimer.toc("plan_compact")
    gtimer.tic("convert_trajectory")
    traj, succ = convert_trajectory(c_traj, joint_num)
    gtimer.toc("convert_trajectory")
print(gtimer)
print(succ)

init_ros: 	88.0 ms/1 = 88.403 ms (88.403/88.403)
add_object: 	0.0 ms/1 = 0.265 ms (0.265/0.265)
plan_compact: 	2283.0 ms/32 = 71.35 ms (27.923/139.322)
convert_trajectory: 	114.0 ms/30 = 3.796 ms (0.033/18.844)
clear_objects: 	0.0 ms/1 = 0.085 ms (0.085/0.085)
add_object2: 	0.0 ms/1 = 0.302 ms (0.302/0.302)

False
